In [3]:
def readin(filename):
    with open(filename) as f:
        lines = f.readlines()

    #something to remove \n 
    str_lines = []
    for l in lines:
        if l[-1:]=="\n":
            str_lines.append(l[0:-1])
        else: 
            str_lines.append(l)

    #check all was fine
    # print(lines[-2:])
    # print(str_lines[-2:])
    return str_lines

elf_data = readin("Day16input.txt")

FileNotFoundError: [Errno 2] No such file or directory: 'Day16input.txt'

# --- Day 16: Proboscidea Volcanium ---

The sensors have led you to the origin of the distress signal: yet another handheld device, just like the one the Elves gave you. However, you don't see any Elves around; instead, the device is surrounded by elephants! They must have gotten lost in these tunnels, and one of the elephants apparently figured out how to turn on the distress signal.

The ground rumbles again, much stronger this time. What kind of cave is this, exactly? You scan the cave with your handheld device; it reports mostly igneous rock, some ash, pockets of pressurized gas, magma... this isn't just a cave, it's a volcano!

You need to get the elephants out of here, quickly. Your device estimates that you have 30 minutes before the volcano erupts, so you don't have time to go back out the way you came in.

You scan the cave for other options and discover a network of pipes and pressure-release valves. You aren't sure how such a system got into a volcano, but you don't have time to complain; your device produces a report (your puzzle input) of each valve's flow rate if it were opened (in pressure per minute) and the tunnels you could use to move between the valves.

There's even a valve in the room you and the elephants are currently standing in labeled AA. You estimate it will take you one minute to open a single valve and one minute to follow any tunnel from one valve to another. What is the most pressure you could release?

In [ ]:
import networkx as nx
import operator as op
import collections as col
import math 
from functools import lru_cache

def mean(pq):
    flowrates = [fr[1] for fr in pq]
    return sum(flowrates)/len(flowrates)

def median(pq):
    flowrates = [fr[1] for fr in pq]
    flowrates.sort()
    midpoint = math.floor(len(flowrates)/2)
    return flowrates[midpoint]

def above_centile(pq, pc):
    flowrates = [fr[1] for fr in pq]
    flowrates.sort()
    point = math.floor(len(flowrates)*pc/100)
    return flowrates[point]

# (a, b) = V_test.gen_priorityQ_list_by_flowrate()
# print(average(a))
# print(average(b))
# print(a)
# print(b)
max_time = 30
start_point = "AA"

def view_graph(V):
    for n in nx.nodes(V):
        print(n, nx.get_node_attributes(V, "flowrate")[n], nx.edges(V, n))

#print(list(nx.get_node_attributes(V_test, "flowrate").items()))


For example, suppose you had the following scan output:

    

In [ ]:
test_data = ["Valve AA has flow rate=0; tunnels lead to valves DD, II, BB",
"Valve BB has flow rate=13; tunnels lead to valves CC, AA",
"Valve CC has flow rate=2; tunnels lead to valves DD, BB",
"Valve DD has flow rate=20; tunnels lead to valves CC, AA, EE",
"Valve EE has flow rate=3; tunnels lead to valves FF, DD",
"Valve FF has flow rate=0; tunnels lead to valves EE, GG",
"Valve GG has flow rate=0; tunnels lead to valves FF, HH",
"Valve HH has flow rate=22; tunnel leads to valve GG",
"Valve II has flow rate=0; tunnels lead to valves AA, JJ",
"Valve JJ has flow rate=21; tunnel leads to valve II"]



order: Ax, D20, Cx, B13, Ax, Ix, J21, Ix, Ax, D20, Ex, Fx, Gx, H22, Gx, Fx, E3, Dx, C2, 

In [ ]:
class Elf_Volcano:

    def __init__(self, edges, labels):
        self.G = nx.Graph()
        #V.add_nodes_from(labels) # actually not needed, any unconnected nodes become unnecessary (unless start node...)
        self.G.add_edges_from(edges)
        nx.set_node_attributes(self.G, labels, "flowrate")
        (self.priorityQ, self.zeros) = self.gen_priorityQ_list_by_flowrate()
        self.accummulated_pressure = 0
        self.sum_open_pressure = 0
        self.current_valve = None
        self.target = None
        self.time_left = -1
        self.current_path = None
        self.open_valves = []
        sp  = {}
        for n in self.G.nodes():
            sp[n] = nx.single_source_shortest_path_length(self.G, n)
        nx.set_node_attributes(self.G, sp, "sps")
        self.option_return = {}

    def get_flow(self, node):
        return nx.get_node_attributes(self.G, "flowrate")[node]
    
    def get_sps(self, node):
        return nx.get_node_attributes(self.G, "sps")[node]

    def get_node(self, node_name):
        return nx.nodes(self.G)[node_name]
    
    def gen_priorityQ_list_by_flowrate(self):
        v_list = [x for x in nx.get_node_attributes(self.G, "flowrate").items() if x[1] > 0]
        zeros = [x for x in nx.get_node_attributes(self.G, "flowrate").items() if x[1] == 0]
        v_list.sort(reverse=True, key = op.itemgetter(1))
        return v_list, zeros

    def maximise_pressure(self, start_point, time_left):
        self.current_valve = start_point
        self.time_left = time_left
        self.open_valves = []
        self.start_set_target()
        while self.time_left>0:
            self.take_turn(time_left)
        return self.accummulated_pressure

    def take_turn(self, time_left):
        if self.time_left>0:
            self.open_valve(time_left)
        if self.time_left>0:
            self.choose_and_move_to_tunnel(time_left)

    def time_tick(self, time_left, msg):
        self.time_left = self.time_left - 1
        self.accummulated_pressure = self.accummulated_pressure + self.sum_open_pressure
        print("== Minute "+ str(time_left-self.time_left)+  " == \n open valves: "+ str(self.open_valves)+" releasing "+str(self.sum_open_pressure)+ msg +"\n")

    def open_valve(self, time_left): 
        flow = nx.get_node_attributes(self.G,"flowrate")[self.current_valve]
        if self.target is not None:
            if flow>0 and self.current_valve not in self.open_valves and self.current_valve == self.target[0]:
                msg = "\nYou open valve "+ self.current_valve
                self.time_tick(time_left, msg)
                self.open_valves.append(self.target)
                self.priorityQ.remove(self.target)
                self.sum_open_pressure = self.sum_open_pressure + self.target[1]
                
    def choose_and_move_to_tunnel(self, time_left):
        self.start_set_target()
        if self.target is not None:
            self.current_path = nx.shortest_path(self.G, self.current_valve, self.target[0])
            #print(self.current_path)
            for n in range(0, len(self.current_path)-1):
                if self.time_left>0:
                    self.current_valve = self.current_path[n+1] #we know there's an edge
                    msg = "\nYou move to "+ str(self.current_valve)
                    self.time_tick(time_left, msg)
        else: 
            msg = ""
            self.time_tick(time_left, msg)

    def start_set_target(self):
        self.set_target(1)


    @lru_cache()
    def set_target(self, distance):
        self.target = None
        while len(self.priorityQ)>0 and self.target is None:
            list_to_check = self.get_neighbours_to_check(distance)
            self.check_nearest(list_to_check)
            if self.target is None: 
                self.set_target(distance+1)

    def get_neighbours_to_check(self, distance):
        sps = nx.get_node_attributes(self.G, "sps")[self.current_valve]
        print(sps)
        list_to_check = [(p, nx.get_node_attributes(self.G, "flowrate")[p]) for p in sps if sps[p] == distance]
        return list_to_check
        # set_targetDP(self.time_left)

            
    def check_nearest(self, to_check):       
        c = [x for x in to_check if x not in (self.open_valves + self.zeros)]
        print(c)
        if len(c)>0:
            candidate = max(c, key=op.itemgetter(1)) 
            threshold = mean(self.priorityQ)
            if candidate[1] >= threshold:
                self.target = candidate


    def start_to_set_targetDP(self):
        ov = tuple(self.open_valves.sort())
        self.target = self.get_node(self.set_targetDP(self.time_left.copy(), self.current_valve.copy(), ov)[2])

    def set_targetDP(self, time, current_valve, open_valves):
        if time == 0: #no moves possible
            return (0, 0, None)

        elif time == 1:
            #either turn the valve you are at
            #or travel 1 step (but it doesn't really matter)
            if self.current_valve not in (open_valves + self.zeros):
                return (self.current_valve.get_flow(), 0, self.current_valve)
            else:
                return (0, 0, None)

        elif time == 2:
            #either open valve then only 1 minute left, can't use
            if self.current_valve not in (list(open_valves) + self.zeros):
                time_after0 = time-(1+0) #-1 to open the valve, can't do anything with the other time tick
                returnP0 = nx.get_node_attributes(self.G, "flowrate")[self.current_valve]+0
                target_temp0 = self.get_node(self.current_valve)
            #or travel 1 step then open valve
                time_after1 = time-(1+1) #-1 to travel, -1 to open the valve on the next minute
                list_to_check = self.get_neighbours_to_check(1)
                best_option = max(list_to_check, key=op.itemgetter(1)) #best option = (nodename, flowrate)
                returnP1 = 0 + best_option[1]  #does this look ok 
                target_temp = best_option[0]

                if returnP1>returnP0:
                    return (returnP1, time_after1, target_temp)
                else: 
                    return (returnP0, time_after0, target_temp)
        
        else:
            #choices:
            #open current valve then epxlore best time-1 
            if self.current_valve not in (self.open_valves + self.zeros):
                #return is 0 this step and current node flowrate the next step
                #option 0 
                valves_after = tuple(list(open_valves).append(self.current_valve).sort())
                time_minus_one = self.set_targetDP(time-1, valves_after, self.current_valve.copy())
                time_after = time - 1
                returnP = 0 + self.current_valve.get_flow() + time_minus_one[0]
                target = self.current_valve

            #travel 1 step then open valve, then look at time -2
            #travel 2 step then open valve, then look at time -3
            #...
            #travel (time-1) steps then open valve, then look at time 0
            for t in range(1, time):
                time_after_temp = time-(t+1)
                list_to_check = self.get_neighbours_to_check(t)
                best_option_temp = max(list_to_check, key=op.itemgetter(1))
                valves_after = open_valves.append(best_option_temp[0])
                returnP_temp = 0 + best_option_temp[1] + self.set_targetDP(time_after_temp, valves_after, best_option_temp[0])
                target_temp = best_option_temp[0]

                if returnP_temp>returnP:
                    returnP = returnP_temp
                    time_after = time_after_temp
                    target = target_temp
            
           # open_valves.sort()
            return (returnP, time_after, target)

            
        #choose maximum
        #return maximum returnP
        



In [ ]:
def parse_data(data):
    labels = {}
    edges = []
    for d in data:
        words = d.split()
        #identify edges
        for e in words[9:]:
            edges.append((words[1], e[0:2]))
        #identify flow rates for valves
        fr = words[4].split("=")
        fr[1] = fr[1].removesuffix(";")
        labels[words[1]] = eval(fr[1])
    V = Elf_Volcano(edges, labels)
    return V

In [ ]:
V_test = parse_data(test_data)
# V_elf = parse_data(elf_data)
print(V_test.maximise_pressure(start_point, max_time))

In [ ]:
V_test_copy = parse_data(test_data)
# print((1, 2) in [(1, 2), (3, 4)])
# V_test_copy.option_return[(1, 2, [])] = (3,7, 2, 7)
# print(V_test_copy.option_return.keys())
# print((1, 2, []) in V_test_copy.option_return.keys())
print(V_test_copy.set_targetDP(2, start_point, []))



All of the valves begin closed. You start at valve AA, but it must be damaged or jammed or something: its flow rate is 0, so there's no point in opening it. However, you could spend one minute moving to valve BB and another minute opening it; doing so would release pressure during the remaining 28 minutes at a flow rate of 13, a total eventual pressure release of 28 * 13 = 364. Then, you could spend your third minute moving to valve CC and your fourth minute opening it, providing an additional 26 minutes of eventual pressure release at a flow rate of 2, or 52 total pressure released by valve CC.



Making your way through the tunnels like this, you could probably open many or all of the valves by the time 30 minutes have elapsed. However, you need to release as much pressure as possible, so you'll need to be methodical. Instead, consider this approach:

    == Minute 1 ==
    No valves are open.
    You move to valve DD.

    == Minute 2 ==
    No valves are open.
    You open valve DD.

    == Minute 3 ==
    Valve DD is open, releasing 20 pressure.
    You move to valve CC.

    == Minute 4 ==
    Valve DD is open, releasing 20 pressure.
    You move to valve BB.

    == Minute 5 ==
    Valve DD is open, releasing 20 pressure.
    You open valve BB.

    == Minute 6 ==
    Valves BB and DD are open, releasing 33 pressure.
    You move to valve AA.

    == Minute 7 ==
    Valves BB and DD are open, releasing 33 pressure.
    You move to valve II.

    == Minute 8 ==
    Valves BB and DD are open, releasing 33 pressure.
    You move to valve JJ.

    == Minute 9 ==
    Valves BB and DD are open, releasing 33 pressure.
    You open valve JJ.

    == Minute 10 ==
    Valves BB, DD, and JJ are open, releasing 54 pressure.
    You move to valve II.

    == Minute 11 ==
    Valves BB, DD, and JJ are open, releasing 54 pressure.
    You move to valve AA.

    == Minute 12 ==
    Valves BB, DD, and JJ are open, releasing 54 pressure.
    You move to valve DD.

    == Minute 13 ==
    Valves BB, DD, and JJ are open, releasing 54 pressure.
    You move to valve EE.

    == Minute 14 ==
    Valves BB, DD, and JJ are open, releasing 54 pressure.
    You move to valve FF.

    == Minute 15 ==
    Valves BB, DD, and JJ are open, releasing 54 pressure.
    You move to valve GG.

    == Minute 16 ==
    Valves BB, DD, and JJ are open, releasing 54 pressure.
    You move to valve HH.

    == Minute 17 ==
    Valves BB, DD, and JJ are open, releasing 54 pressure.
    You open valve HH.

    == Minute 18 ==
    Valves BB, DD, HH, and JJ are open, releasing 76 pressure.
    You move to valve GG.

    == Minute 19 ==
    Valves BB, DD, HH, and JJ are open, releasing 76 pressure.
    You move to valve FF.

    == Minute 20 ==
    Valves BB, DD, HH, and JJ are open, releasing 76 pressure.
    You move to valve EE.

    == Minute 21 ==
    Valves BB, DD, HH, and JJ are open, releasing 76 pressure.
    You open valve EE.

    == Minute 22 ==
    Valves BB, DD, EE, HH, and JJ are open, releasing 79 pressure.
    You move to valve DD.

    == Minute 23 ==
    Valves BB, DD, EE, HH, and JJ are open, releasing 79 pressure.
    You move to valve CC.

    == Minute 24 ==
    Valves BB, DD, EE, HH, and JJ are open, releasing 79 pressure.
    You open valve CC.

    == Minute 25 ==
    Valves BB, CC, DD, EE, HH, and JJ are open, releasing 81 pressure.

    == Minute 26 ==
    Valves BB, CC, DD, EE, HH, and JJ are open, releasing 81 pressure.

    == Minute 27 ==
    Valves BB, CC, DD, EE, HH, and JJ are open, releasing 81 pressure.

    == Minute 28 ==
    Valves BB, CC, DD, EE, HH, and JJ are open, releasing 81 pressure.

    == Minute 29 ==
    Valves BB, CC, DD, EE, HH, and JJ are open, releasing 81 pressure.

    == Minute 30 ==
    Valves BB, CC, DD, EE, HH, and JJ are open, releasing 81 pressure.

This approach lets you release the most pressure possible in 30 minutes with this valve layout, 1651.

Work out the steps to release the most pressure in 30 minutes. What is the most pressure you can release?

*2454 is too high, 1885 and 1756 are too low*
*1981 is not right*

In [ ]:
V_elf = parse_data(elf_data)
print(start_point, max_time)
print("RESULT = ",V_elf.maximise_pressure(start_point, max_time))